In [1]:
import pandas as pd

In [2]:
year1 = 2021
freq_threshold = 0.001
covar_path = '/share/garg/311_data/sb2377/clean_codebase/tract_demographics.csv'
year1_path = '/share/garg/311_data/sb2377/clean_codebase/{}_weekly_report_data_tracts.csv'.format(year1)
two_year_path = '/share/garg/311_data/sb2377/clean_codebase/two_year_reports_full.h5'
graph_path = '/share/garg/311_data/sb2377/clean_codebase/nyc_census_graph.csv'
base_df_save_path = '/share/garg/311_data/sb2377/clean_codebase/three_year_base.csv'
full_df_save_path = '/share/garg/311_data/sb2377/clean_codebase/three_year_reports_full.h5'

In [3]:
covariates_arr = pd.read_csv(covar_path)
df_year1 = pd.read_csv(year1_path)
df_year2 = pd.read_hdf(two_year_path, 'df')
df_year1 = df_year1.loc[:, ~df_year1.columns.str.contains('Unnamed', case=False)]
df_year2 = df_year2[df_year1.columns]
graph_data_df = pd.read_csv(graph_path)

In [4]:
# merge three food related complaint types (for food rating data)
reports1 = df_year1[df_year1['typeagency'] == 'FoodEstablishmentDOHMH'][['GEOID', 'report_week', 'reported']]
reports2 = df_year1[df_year1['typeagency'] == 'MobileFoodVendorDOHMH'][['GEOID', 'report_week', 'reported']]
reports3 = df_year1[df_year1['typeagency'] == 'FoodPoisoningDOHMH'][['GEOID', 'report_week', 'reported']]
merged_df = reports1.merge(reports2, on=['GEOID', 'report_week'], suffixes=('_1', '_2')).merge(reports3, on=['GEOID', 'report_week'])
merged_df.rename(columns={'reported': 'reported_3'}, inplace=True)
merged_df['reported_1'] = merged_df['reported_1'].astype('bool')
merged_df['reported_2'] = merged_df['reported_2'].astype('bool')
merged_df['reported_3'] = merged_df['reported_3'].astype('bool')
merged_df['reported'] = merged_df['reported_1'] | merged_df['reported_2'] | merged_df['reported_3']
food_df = df_year1[df_year1['typeagency'] == 'FoodEstablishmentDOHMH'].copy()
food_df['typeagency'] = 'FoodDOHMH'
food_df = food_df.drop('reported', axis=1)
food_df = pd.merge(food_df, merged_df[['GEOID', 'report_week', 'reported']], on=['GEOID', 'report_week'])
df_year1 = df_year1[~df_year1['typeagency'].isin(['FoodEstablishmentDOHMH', 'MobileFoodVendorDOHMH', 'FoodPoisoningDOHMH'])]
df_year1 = pd.concat([df_year1, food_df])

In [5]:
covars = ['normalized_log_population_density', 
          'normalized_log_population', 
          'normalized_log_income_median',
          'normalized_education_bachelors_pct', 
          'normalized_race_white_nh_pct',
          'normalized_age_median', 
          'normalized_households_renteroccupied_pct']

In [6]:
# only get types that appear in both years
types_year1 = set(df_year1['typeagency'].unique())
types_year2 = set(df_year2['typeagency'].unique())
type_intersection = types_year1.intersection(types_year2)
df_year1 = df_year1[df_year1['typeagency'].isin(type_intersection)].copy()
df_year2 = df_year2[df_year2['typeagency'].isin(type_intersection)].copy()

In [7]:
# merge reports for the last week of year 1 and the first week of year 2
# 'week' = week number across both years
# 'week_of_year' = week number within year
df_year2['report_week'] = df_year2['report_week'] + 52

# combine reports for the last week of year 1 and the first week of year 2
df_year1['reported'] = df_year1['reported'].astype(int)
df_year2['reported'] = df_year2['reported'].astype(int)

# last week year 1
reported_year1 = df_year1[df_year1['report_week'] == 52][['GEOID', 'typeagency', 'reported']]

# first week year 2
reported_year2 = df_year2[df_year2['report_week'] == 52][['GEOID', 'typeagency', 'reported']]

# combine
merged_reports = pd.merge(reported_year1, reported_year2, on=['GEOID', 'typeagency'], how='outer', suffixes=('_year1', '_year2'))
merged_reports.fillna(0, inplace=True)
merged_reports['reported_year1'] = merged_reports['reported_year1'].astype(bool)
merged_reports['reported_year2'] = merged_reports['reported_year2'].astype(bool)
merged_reports['reported'] = merged_reports['reported_year1'] | merged_reports['reported_year2']
merged_reports = merged_reports.drop(columns=['reported_year1', 'reported_year2'])

# add to year 1
df_year1_week = df_year1[df_year1['report_week'] == 52]
df_year1_week = df_year1_week.drop(columns=['reported'])
df_year1_week = pd.merge(df_year1_week, merged_reports, on=['GEOID', 'typeagency'])
df_year1 = pd.concat([df_year1[df_year1['report_week'] != 52], df_year1_week], axis=0)

# combine both years of data
df_year2 = df_year2[df_year2['report_week'] != 52]
df = pd.concat([df_year1, df_year2])

In [8]:
print('num nodes = {}'.format(len(df['GEOID'].unique())))
print('num types = {}'.format(len(df['typeagency'].unique())))
print('num weeks = {}'.format(len(df['report_week'].unique())))
print(len(df))

num nodes = 2326
num types = 140
num weeks = 157
50630440


In [9]:
# get rows with valid census tracts
valid_GEOIDs = covariates_arr['GEOID'].unique()
df = df[df['GEOID'].isin(valid_GEOIDs)]

# get covariates
df = pd.merge(df, covariates_arr, on='GEOID')

# filter out types with with very small frequencies
type_pT = df.groupby(['typeagency'])['reported'].mean()
type_to_include_df = type_pT[(type_pT > freq_threshold)]
included_types = type_to_include_df.index.tolist()
subset_df = df[(df['typeagency'].isin(included_types))]
print('percentage of data kept = {:.2f}%'.format(len(subset_df) / len(df) * 100))
print('percentage of types kept = {:.2f}%'.format(len(included_types) / len(type_pT) * 100))

percentage of data kept = 99.29%
percentage of types kept = 99.29%


In [10]:
# map each node and type to a unique indexer
nodes = subset_df['GEOID'].unique()
num_nodes = len(nodes)
types = subset_df['typeagency'].unique()
num_types = len(types)
node_mapping = {old_id: new_id for new_id, old_id in enumerate(nodes)}
type_mapping = {old_id: new_id for new_id, old_id in enumerate(types)}
subset_df['node_idxs'] = subset_df['GEOID'].map(node_mapping).values
subset_df['type_idxs'] = subset_df['typeagency'].map(type_mapping).values
print('num nodes = {}'.format(num_nodes))
print('num types = {}'.format(num_types))

/share/pierson/tmp_directory_location_please_read_readme/sb2377_tmp/ipykernel_4082275/33382678.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['node_idxs'] = subset_df['GEOID'].map(node_mapping).values


num nodes = 2292
num types = 139


/share/pierson/tmp_directory_location_please_read_readme/sb2377_tmp/ipykernel_4082275/33382678.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['type_idxs'] = subset_df['typeagency'].map(type_mapping).values


In [11]:
assert(len(subset_df[subset_df.isna().any(axis=1)]) == 0)

In [12]:
# create base df
base_df = subset_df[['GEOID', 'typeagency', 'Complaint Type', 'Agency', 'node_idxs', 'type_idxs'] + covars].drop_duplicates()
base_df.to_csv(base_df_save_path)

In [13]:
assert(len(base_df[base_df.isna().any(axis=1)]) == 0)

In [14]:
subset_df.to_hdf(full_df_save_path, key='df', mode='w')